In [ ]:
#판다스 임포트
import pandas as pd

#csv 파일 읽기
df_bond = pd.read_csv('\\final_Data\\total_ngram.csv', encoding='utf-8', engine='python')





In [ ]:
#Pos n-gram 추출
list_up=[]
for ngram in df_bond[df_bond['label']=='상승']['n-gram']:
    list_up.extend(ngram.split(','))
    
#Neg n-gram 추출
list_down=[]
for ngram in df_bond[df_bond['label']=='하락']['n-gram']:
    list_down.extend(ngram.split(','))
    


In [ ]:
    #Pos n-gram 중 15개 이하인 값들 제거
tmp_df = pd.DataFrame(list_up, columns=['상승'])
up_df=pd.DataFrame(tmp_df['상승'].value_counts())
up_df=up_df[up_df['상승']>=15]
up_df

In [ ]:
#Neg n-gram 중 15개 이하인 값들 제거
tmp_df = pd.DataFrame(list_down, columns=['하락'])
down_df=pd.DataFrame(tmp_df['하락'].value_counts())
down_df=down_df[down_df['하락']>=15]
down_df

In [ ]:
#n-gram을 기준으로 합병하기 위해 인덱스 칼럼으로 뺌
up_df=up_df.reset_index()
down_df=down_df.reset_index()

#병합.
updown_df=pd.merge(up_df,down_df,how='outer') #outer로 하여 한쪽만 있는 값이 있어도 데이터프레임에 삽입할 수 있게함.
updown_df=updown_df.rename({'index':'n-gram'},axis=1)
updown_df=updown_df.set_index('n-gram')
updown_df=updown_df.fillna(1) #nan 값을 1로 채움
updown_df=updown_df.sort_values(['상승'],ascending=[False]) #pos 기준으로 내림차순

In [ ]:
#총 Pos, Neg 계산
tot_pos=sum(updown_df['상승'])
tot_neg=sum(updown_df['하락'])

In [ ]:
#NBC 칼럼 추가, NBC 값 추가
updown_df.loc[:,'nbc']=0

for idx in updown_df.index:
    updown_df.loc[idx,'nbc']=(updown_df.loc[idx,'상승']/tot_pos)/(updown_df.loc[idx,'하락']/tot_neg)

In [ ]:

# 0.76 < x < 1.3 값 제거
# nbc_df=updown_df[(updown_df['nbc']<=0.76)|(updown_df['nbc']>=1.3)]
#제거안한 전체버전은 이거를 주석처리 해제하시오
nbc_df=updown_df
nbc_df=nbc_df.sort_values(['nbc'],ascending=[False])
nbc_df

In [ ]:
nbc_df.to_csv('polarity.csv', encoding='utf-8')